#Apprentissage par Renforcement pour la Collecte de Déchets avec ROS - TEKBOT Robotics Challenge 2025

Dans ce notebook, nous allons apprendre à entraîner un robot mobile autonome à collecter des déchets dans un environnement simulé. L'agent sera entraîné avec l'apprentissage par renforcement (RL) et pourra s’intégrer dans un système robotique via ROS.

Ce tutoriel s’adresse aux débutants/intermédiaires en RL et ROS.

## Objectif
Former un agent à :
- Naviguer dans un environnement
- Éviter les obstacles
- Collecter les déchets

## Technologies utilisées :
- Python
- OpenAI Gym (ou gymnasium)
- Stable-Baselines3
- ROS (via rosbridge ou gym-gazebo)

## Implémentation :
La construction de l'environnemt et l'entrainement se fera directement dans votre module ROS


# Préparation de l'environnement

In [ ]:
!pip install gym stable-baselines3[extra] matplotlib numpy
# Optionnel ROS via Jupyter : !pip install rospkg catkin_pkg
import gym
import numpy as np
import matplotlib.pyplot as plt
from stable_baselines3 import PPO

## 🤖 Définition de l’environnement personnalisé

Si tu ne veux pas démarrer avec Gazebo tout de suite, commence avec une version 2D type GridWorld :

In [ ]:
from gym import spaces
from gym.utils import seeding

class WasteCollectionEnv(gym.Env):
    def __init__(self, grid_size=5, n_dechets=3):
        self.grid_size = grid_size
        self.agent_pos = [0, 0]
        self.dechets = []
        self.action_space = spaces.Discrete(4)  # N, S, E, W
        self.observation_space = spaces.Box(low=0, high=1, shape=(grid_size, grid_size, 1), dtype=np.uint8)
        self._reset()

    def _reset(self):
        self.agent_pos = [0, 0]
        self.dechets = [list(np.random.randint(0, self.grid_size, size=2)) for _ in range(3)]
        return self._get_obs()

    def _get_obs(self):
        obs = np.zeros((self.grid_size, self.grid_size, 1), dtype=np.uint8)
        obs[self.agent_pos[0], self.agent_pos[1]] = 1
        for d in self.dechets:
            obs[d[0], d[1]] = 2
        return obs

    def step(self, action):
        x, y = self.agent_pos
        if action == 0 and x > 0: x -= 1
        if action == 1 and x < self.grid_size - 1: x += 1
        if action == 2 and y < self.grid_size - 1: y += 1
        if action == 3 and y > 0: y -= 1
        self.agent_pos = [x, y]

        reward = 0
        done = False
        if self.agent_pos in self.dechets:
            reward = 1
            self.dechets.remove(self.agent_pos)
        if len(self.dechets) == 0:
            done = True

        return self._get_obs(), reward, done, {}

    def render(self):
        grid = np.zeros((self.grid_size, self.grid_size))
        for d in self.dechets:
            grid[d[0], d[1]] = 0.5
        grid[self.agent_pos[0], self.agent_pos[1]] = 1.0
        print(grid)

## 🤖  Entraînement avec PPO

In [ ]:
from stable_baselines3 import PPO
from stable_baselines3.common.env_checker import check_env

env = WasteCollectionEnv()
check_env(env, warn=True)

model = PPO("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=10_000)

## 📈 Évaluation

In [ ]:
obs = env.reset()
total_reward = 0
for _ in range(20):
    action, _ = model.predict(obs)
    obs, reward, done, info = env.step(action)
    total_reward += reward
    env.render()
    if done:
        break
print(f"Total reward collected: {total_reward}")

## Intégration avec ROS

Pour passer du monde simulé à ROS :

- L’environnement peut être recréé dans Gazebo
- L’agent peut publier des commandes (Twist) sur `/cmd_vel`
- Les capteurs (caméra, lidar, odom) peuvent être consommés via ROS Bridge

### Approches possibles :

- Utiliser `gym-gazebo2` pour connecter RL à ROS2
- Utiliser `rosbridge_suite` pour interagir via WebSocket avec Python
- Utiliser un nœud ROS Python (`rospy`) pour connecter le modèle

> Exemple de code ROS pour envoyer l’action du modèle :
```python
import rospy
from geometry_msgs.msg import Twist

pub = rospy.Publisher('/cmd_vel', Twist, queue_size=10)
rospy.init_node('rl_agent_node')

twist = Twist()
twist.linear.x = 0.2  # exemple simple
twist.angular.z = 0.0
pub.publish(twist)

    Pour connecter le modèle RL :

    charger le modèle entraîné avec model = PPO.load('model_path')

    transformer observation ROS → format d’observation RL

    prédire l’action et publier

Tu peux aussi utiliser ROS2 + Gazebo + gym-gazebo2 si tu veux faire une simulation plus réaliste.


---

### Conclusion
Ce tutoriel montre comment :
- Créer un environnement d’apprentissage par renforcement
- Entraîner un agent avec Stable-Baselines3
- Intégrer ROS pour passer en environnement réel ou simulé

🎯 Prochaine étape :
- Tester dans Gazebo
- Ajouter des obstacles
- Gérer plusieurs types de déchets (recyclables, non-recyclables)


## Explication sur la construction de l'environnement
Nous définissons un environnement d’apprentissage par renforcement personnalisé pour simuler la **collecte de déchets dans une grille**. Il est compatible avec `gym`, une bibliothèque utilisée pour entraîner des agents d’apprentissage par renforcement.

---

## 📦 Analyse ligne par ligne

### 🧩 Importations

```python
from gym import spaces
from gym.utils import seeding
```

* `spaces`: permet de définir les **espaces d’actions et d’observations**.
* `seeding`: pour le contrôle de l’aléatoire (non utilisé ici, mais utile pour reproductibilité).

---

### 🧪 Définition de l’environnement

```python
class WasteCollectionEnv(gym.Env):
```

Cette classe hérite de `gym.Env` et doit donc implémenter les méthodes standards : `step()`, `reset()`, `render()`, etc.

---

### 🔧 Initialisation (`__init__`)

```python
def __init__(self, grid_size=5, n_dechets=3):
```

* `grid_size=5` → taille de la grille carrée.
* `n_dechets=3` → nombre de déchets à collecter (non utilisé directement ici, c’est codé en dur à 3).

```python
self.grid_size = grid_size
self.agent_pos = [0, 0]  # position de départ
self.dechets = []        # liste des positions de déchets
```

#### 🧠 Espace d’action

```python
self.action_space = spaces.Discrete(4)
```

* 4 actions possibles : `0 = haut`, `1 = bas`, `2 = droite`, `3 = gauche`

#### 👁️ Espace d’observation

```python
self.observation_space = spaces.Box(low=0, high=1, shape=(grid_size, grid_size, 1), dtype=np.uint8)
```

* Observation : une **grille (image 2D)** où :

  * `0` = vide
  * `1` = agent
  * `2` = déchet

---

### 🧼 Réinitialisation

```python
def _reset(self):
    self.agent_pos = [0, 0]
    self.dechets = [list(np.random.randint(0, self.grid_size, size=2)) for _ in range(3)]
    return self._get_obs()
```

* Replace l’agent au coin `[0, 0]`
* Place 3 déchets à des positions aléatoires
* Retourne l'observation actuelle

---

### 👀 Observation

```python
def _get_obs(self):
    obs = np.zeros((self.grid_size, self.grid_size, 1), dtype=np.uint8)
    obs[self.agent_pos[0], self.agent_pos[1]] = 1
    for d in self.dechets:
        obs[d[0], d[1]] = 2
    return obs
```

* Crée une **grille vide**
* Place l’agent et les déchets dans cette grille

---

### 🔁 Étape d’interaction

```python
def step(self, action):
```

* Met à jour la position de l’agent en fonction de l’action choisie :

```python
if action == 0 and x > 0: x -= 1   # haut
if action == 1 and x < self.grid_size - 1: x += 1  # bas
if action == 2 and y < self.grid_size - 1: y += 1  # droite
if action == 3 and y > 0: y -= 1   # gauche
```

* Si l’agent atteint un déchet :

```python
if self.agent_pos in self.dechets:
    reward = 1
    self.dechets.remove(self.agent_pos)
```

* Si tous les déchets sont ramassés, l’épisode se termine :

```python
if len(self.dechets) == 0:
    done = True
```

---

### 🖼️ Affichage console

```python
def render(self):
    grid = np.zeros((self.grid_size, self.grid_size))
    for d in self.dechets:
        grid[d[0], d[1]] = 0.5
    grid[self.agent_pos[0], self.agent_pos[1]] = 1.0
    print(grid)
```

* `0.0` → case vide
* `0.5` → déchet
* `1.0` → agent

---

## ✅ Exemple d’utilisation

```python
env = WasteCollectionEnv()
obs = env._reset()
env.render()

obs, reward, done, _ = env.step(2)  # Move right
env.render()
```

---

## 🎯 Conclusion

Ce code est un **environnement simplifié en grille** pour entraîner un agent à **se déplacer et ramasser des déchets**. Il peut facilement être :

* Étendu à ROS/Gazebo
* Connecté à un modèle PPO/DQN


Utiliser **Gazebo avec ROS 2** dans un notebook pour illustrer l’apprentissage par renforcement (RL) dans la collecte de déchets est très pertinent et professionnalisant.

---

## ✅ Objectif

Construire un **notebook tutoriel** qui :

1. Crée une simulation Gazebo de robot collecteur de déchets (turtlebot ou modèle personnalisé)
2. Entraîne un agent RL (PPO) dans un environnement `gym-gazebo2`
3. Connecte ROS 2 pour publier les commandes (via `ros2_control` ou `/cmd_vel`)
4. Fait une boucle observation → action → ROS

---

## 💡 Architecture du Projet

```
.
├── notebook_rl_waste_collector.ipynb  <-- Notebook tutoriel
├── gazebo_env/
│   ├── models/                        <-- Déchets et obstacles
│   ├── worlds/                        <-- Monde simulé
│   └── launch/
│       └── waste_world.launch.py     <-- Lancement de Gazebo + ROS 2
├── rl/
│   └── waste_collector_env.py        <-- Classe GymEnv pour ROS2
└── install.sh                        <-- Script d'installation
```

---

## 📦 Étapes Clés du Notebook

---

### 🧪 Étape 1 : Setup (hors notebook)

Avant de pouvoir tout exécuter dans un notebook, il faut **préparer le système** :

### ✅ Installation de ROS 2 + Gazebo + gym-gazebo2

```bash
# Pour Ubuntu 22.04
sudo apt update
sudo apt install ros-humble-desktop python3-colcon-common-extensions ros-humble-gazebo-ros-pkgs

# Créer un workspace
mkdir -p ~/rl_ws/src
cd ~/rl_ws/src

# Cloner un environnement de simulation (ex: turtlebot3 + gym-gazebo2)
git clone https://github.com/ros-simulation/gym-gazebo2.git
cd ..
colcon build --symlink-install
source install/setup.bash
```

---

### 📦 Étape 2 : Lancement du monde Gazebo

```python
# Code dans une cellule shell de notebook Jupyter (optionnel)
!ros2 launch gym_gazebo2 turtlebot3_world.launch.py
```

Sinon dans un terminal :

```bash
ros2 launch gym_gazebo2 turtlebot3_world.launch.py
```

---

### 🔁 Étape 3 : Création d’un wrapper Gym pour l’environnement ROS 2

```python
# rl/waste_collector_env.py

import gym
import numpy as np
import rclpy
from rclpy.node import Node
from geometry_msgs.msg import Twist
from nav_msgs.msg import Odometry

class WasteCollectorEnv(gym.Env):
    def __init__(self):
        super().__init__()
        rclpy.init()
        self.node = rclpy.create_node("rl_agent")

        self.pub = self.node.create_publisher(Twist, "/cmd_vel", 10)
        self.odom_sub = self.node.create_subscription(Odometry, "/odom", self.odom_callback, 10)
        self.pose = None

        self.action_space = gym.spaces.Discrete(4)  # N, S, E, W or FWD, LEFT, RIGHT, STOP
        self.observation_space = gym.spaces.Box(low=0, high=1, shape=(5,), dtype=np.float32)

    def odom_callback(self, msg):
        self.pose = msg.pose.pose

    def step(self, action):
        twist = Twist()
        if action == 0:
            twist.linear.x = 0.3
        elif action == 1:
            twist.angular.z = 0.5
        elif action == 2:
            twist.angular.z = -0.5
        elif action == 3:
            twist.linear.x = 0.0

        self.pub.publish(twist)
        rclpy.spin_once(self.node, timeout_sec=0.1)

        obs = self._get_observation()
        reward = self._compute_reward()
        done = False  # Ajoute logique selon objectif atteint
        return obs, reward, done, {}

    def _get_observation(self):
        if self.pose:
            return np.array([
                self.pose.position.x,
                self.pose.position.y,
                self.pose.orientation.z,
                self.pose.orientation.w,
                1.0  # Distance to nearest "trash" object, à calculer
            ])
        return np.zeros(5)

    def _compute_reward(self):
        # À personnaliser : proximité avec les déchets
        return 1.0 if self.pose and self.pose.position.x > 1.0 else 0.0

    def reset(self):
        # Reset via ROS service si nécessaire
        return self._get_observation()
```

---

### 🧠 Étape 4 : Entraînement RL dans le Notebook

```python
from stable_baselines3 import PPO
from rl.waste_collector_env import WasteCollectorEnv

env = WasteCollectorEnv()
model = PPO("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=50000)
model.save("ppo_waste_collector")
```

---

### 📊 Étape 5 : Déploiement en conditions Gazebo

```python
# Charger et exécuter
model = PPO.load("ppo_waste_collector")

obs = env.reset()
for i in range(200):
    action, _states = model.predict(obs)
    obs, reward, done, info = env.step(action)
    if done:
        break
```

---

## 🔌 Bonus : Intégration ROS 2 complète

Si tu veux aller plus loin :

* Crée un nœud ROS 2 `rl_agent_node.py` qui importe le modèle entraîné
* Publie directement sur `/cmd_vel` à chaque prédiction
* Abonne-toi à `/odom` et éventuellement `/camera` pour extraire les observations

---

## 📁 Fichier `waste_world.world` (exemple simple Gazebo)

```xml
<world name="waste_world">
  <include>
    <uri>model://ground_plane</uri>
  </include>
  <include>
    <uri>model://turtlebot3_burger</uri>
  </include>
  <include>
    <uri>model://trash_can</uri>
    <pose>1 2 0 0 0 0</pose>
  </include>
</world>
```

---


Lorsqu'on **entraîne un agent avec l'apprentissage par renforcement (RL)**, l’objectif final est souvent d’**exporter un modèle entraîné** afin de l’utiliser dans un système réel comme un robot (via **ROS2 + Gazebo**, par exemple).

---

## 🔁 Pipeline général d’un agent RL

1. 📦 **Définir l’environnement (`env`)** — comme celui que tu as codé.
2. 🧠 **Entraîner un agent** (ex. avec `Stable-Baselines3`, `RLlib`, etc.)
3. 💾 **Sauvegarder le modèle**
4. 🚀 **Charger le modèle pour l’utiliser en production ou dans un simulateur comme Gazebo**

---

## 🧠 Exemple avec `Stable-Baselines3`

### 1. Entraînement de l’agent

```python
from stable_baselines3 import PPO
from stable_baselines3.common.env_checker import check_env

env = WasteCollectionEnv()
check_env(env)  # vérifie que l’environnement est compatible

model = PPO("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=10000)
```

---

### 2. Sauvegarde du modèle

```python
model.save("ppo_waste_collector")
```

Cela va créer un dossier ou fichier contenant :

* Les **poids du modèle**
* La **structure du réseau de neurones**
* Les **hyperparamètres**

---

### 3. Chargement du modèle pour utilisation

```python
from stable_baselines3 import PPO

model = PPO.load("ppo_waste_collector")

obs = env.reset()
done = False
while not done:
    action, _states = model.predict(obs)
    obs, reward, done, info = env.step(action)
    env.render()
```

---

## 🧩 Intégration dans ROS2 + Gazebo

Une fois le modèle entraîné :

### 🔁 Option 1 – Utiliser en Python dans ROS2

Tu peux créer un **nœud ROS2 en Python** qui :

* Charge le modèle
* Récupère l’état du robot via les topics `/odom`, `/scan`, etc.
* Produit une action avec `model.predict(obs)`
* Envoie une commande sur `/cmd_vel` ou autre

```python
import rclpy
from rclpy.node import Node
from geometry_msgs.msg import Twist
from stable_baselines3 import PPO
import numpy as np

class RLController(Node):
    def __init__(self):
        super().__init__('rl_controller')
        self.model = PPO.load("ppo_waste_collector")
        self.publisher = self.create_publisher(Twist, '/cmd_vel', 10)
        self.timer = self.create_timer(0.1, self.control_loop)

    def control_loop(self):
        obs = self.get_observation_from_sensors()
        action, _ = self.model.predict(obs)
        cmd = self.convert_action_to_cmd_vel(action)
        self.publisher.publish(cmd)
```

---

### 📁 Option 2 – Exporter vers ONNX ou TorchScript

Tu peux exporter ton modèle pour l’intégrer dans un autre langage ou déployer sur des systèmes embarqués :

#### 🔷 Export vers ONNX

```python
import torch

model.policy.to("cpu")
dummy_input = torch.randn(1, model.observation_space.shape[0])
torch.onnx.export(model.policy, dummy_input, "waste_policy.onnx")
```

Ce fichier `.onnx` peut ensuite être utilisé avec :

* **ROS2 + ONNX Runtime**
* **NVIDIA Jetson (TensorRT)**
* **Unity / Web / Mobile**

---

## 🚀 Résumé

| Étape        | Objectif               | Code clé                     |
| ------------ | ---------------------- | ---------------------------- |
| Entraînement | Apprendre la stratégie | `model.learn(...)`           |
| Sauvegarde   | Stocker le modèle      | `model.save("...")`          |
| Chargement   | Réutiliser             | `PPO.load("...")`            |
| Déploiement  | ROS2, Gazebo, Jetson   | `model.predict(obs)` ou ONNX |

---

Souhaites-tu un **exemple complet de nœud ROS2** qui charge le modèle entraîné et contrôle un robot dans Gazebo ?
